In [1]:
import os
import subprocess
import json
import shutil
from dotenv import load_dotenv
from IPython.display import display, Markdown, Javascript
import pandas as pd

# Load environment variables (for Neo4j credentials primarily)
load_dotenv()

# --- Configuration ---
ANGULAR_REPO_URL = "https://github.com/gothinkster/angular-realworld-example-app.git"
ANGULAR_PROJECT_DIR_NAME = "angular-realworld-example-app" # the actual directory name
BASE_DIR = os.getcwd() 

# CORRECTED PATH:
ANGULAR_PROJECT_PATH = os.path.join(BASE_DIR, ANGULAR_PROJECT_DIR_NAME)

PARSER_DIR = os.path.join(BASE_DIR, "angular_ast_parser")
PYTHON_SCRIPTS_DIR = os.path.join(BASE_DIR, "python_scripts")
OUTPUT_DIR = os.path.join(BASE_DIR, "output")
PARSED_DATA_FILE = os.path.join(OUTPUT_DIR, "parsed_angular_data.json")

# Neo4j connection details (used for display, actual connection is in scripts)
NEO4J_URI = os.getenv("NEO4J_URI", "bolt://localhost:7687")
NEO4J_USER = os.getenv("NEO4J_USER", "neo4j")
NEO4J_BROWSER_URL = NEO4J_URI.replace("bolt://", "http://").replace(":7687", ":7474/browser/")

print(f"Base directory: {BASE_DIR}")
print(f"Angular project path: {ANGULAR_PROJECT_PATH}")
print(f"Parser directory: {PARSER_DIR}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Neo4j URI (for scripts): {NEO4J_URI}")
print(f"Neo4j Browser: {NEO4J_BROWSER_URL}")

# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

Base directory: /home/radhikesh/Documents/chubb/angular-project-2
Angular project path: /home/radhikesh/Documents/chubb/angular-project-2/angular-realworld-example-app
Parser directory: /home/radhikesh/Documents/chubb/angular-project-2/angular_ast_parser
Output directory: /home/radhikesh/Documents/chubb/angular-project-2/output
Neo4j URI (for scripts): bolt://localhost:7687
Neo4j Browser: http://localhost:7474/browser/


### Clone Angular Repository (if needed)

In [2]:
def clone_repo(repo_url, target_dir_name_in_base):
    # target_dir is now the full path to where the repo should be
    target_dir = os.path.join(BASE_DIR, target_dir_name_in_base)
    
    if os.path.exists(target_dir):
        print(f"Target directory {target_dir} already exists. Skipping clone.")
        # Optional: Add logic to pull latest changes if directory exists
        # try:
        #     print(f"Pulling latest changes for {target_dir}...")
        #     subprocess.run(["git", "pull"], cwd=target_dir, check=True, capture_output=True, text=True)
        # except subprocess.CalledProcessError as e:
        #     print(f"Error pulling latest changes: {e.stderr}")
        return
    
    # The parent directory for cloning is BASE_DIR, and git will create the target_dir_name_in_base directory.
    print(f"Cloning {repo_url} into {target_dir}...")
    try:
        # Git clone will create a directory named after the repo, or you can specify the name
        # We are specifying the target directory name to be ANGULAR_PROJECT_DIR_NAME
        result = subprocess.run(["git", "clone", repo_url, target_dir_name_in_base], cwd=BASE_DIR, check=True, capture_output=True, text=True)
        print("Repository cloned successfully.")
        print(result.stdout)
    except subprocess.CalledProcessError as e:
        print(f"Error cloning repository: {e.stderr}")
        # Consider raising the error or handling it more gracefully
        raise

# Pass the desired directory name for the clone directly under BASE_DIR
clone_repo(ANGULAR_REPO_URL, ANGULAR_PROJECT_DIR_NAME)

Target directory /home/radhikesh/Documents/chubb/angular-project-2/angular-realworld-example-app already exists. Skipping clone.


### Inspect Parsed Data

In [4]:
if os.path.exists(PARSED_DATA_FILE):
    with open(PARSED_DATA_FILE, 'r') as f:
        parsed_data = json.load(f)
    print(f"Loaded parsed data. Number of nodes extracted: {len(parsed_data.get('nodes', []))}")
    if parsed_data.get('nodes'):
        display(pd.DataFrame(parsed_data['nodes'][:5]))
else:
    print(f"{PARSED_DATA_FILE} not found. Skipping inspection.")

Loaded parsed data. Number of nodes extracted: 75


,id,type,name,filePath,relationships,properties
0,File:src/app/core/auth/services/jwt.service.ts,File,jwt.service.ts,src/app/core/auth/services/jwt.service.ts,"[{'type': 'IMPORTS', 'targetId': 'External:@an...",NaN
1,Service:JwtService:src/app/core/auth/services/...,Service,JwtService,src/app/core/auth/services/jwt.service.ts,[],{'providedIn': 'root'}
2,File:src/app/core/auth/user.model.ts,File,user.model.ts,src/app/core/auth/user.model.ts,[],NaN
3,Interface:User:src/app/core/auth/user.model.ts,Interface,User,src/app/core/auth/user.model.ts,[],NaN
4,File:src/app/core/auth/services/user.service.ts,File,user.service.ts,src/app/core/auth/services/user.service.ts,"[{'type': 'IMPORTS', 'targetId': 'External:@an...",NaN


## 2. Load Data into Neo4j

Run the Python script to populate the Neo4j database. Ensure Neo4j server is running.

In [5]:
def run_neo4j_loader(python_scripts_path, parsed_data_file, clear_db=True):
    print("Running Neo4j loader...")
    if not os.path.exists(parsed_data_file):
        print(f"Error: Parsed data file {parsed_data_file} not found. Cannot load into Neo4j.")
        return False
        
    loader_script = os.path.join(python_scripts_path, "neo4j_loader.py")
    command = ["python", loader_script, "--file", parsed_data_file]
    if clear_db:
        command.append("--clear")
    
    try:
        print(f"Executing Neo4j loader command: {' '.join(command)}")
        result = subprocess.run(command, check=True, capture_output=True, text=True, cwd=BASE_DIR)
        print("Neo4j loader finished.")
        print("--- Loader STDOUT ---")
        print(result.stdout)
        if result.stderr:
            print("--- Loader STDERR ---")
            print(result.stderr)
        return True
    except subprocess.CalledProcessError as e:
        print(f"Neo4j loader script failed: {e}")
        print(f"Stdout:\n{e.stdout}")
        print(f"Stderr:\n{e.stderr}")
        return False
    except FileNotFoundError:
        print(f"Error: Python interpreter or script {loader_script} not found.")
        return False

# Set clear_db=True to wipe Neo4j before loading. Set to False to append/update.
if not run_neo4j_loader(PYTHON_SCRIPTS_DIR, PARSED_DATA_FILE, clear_db=True):
    display(Markdown("<font color='red'>**Neo4j data loading failed. Check logs above.**</font>"))
else:
    display(Markdown(f"<font color='green'>**Neo4j data loading completed.**</font> You can inspect the graph at [{NEO4J_BROWSER_URL}]({NEO4J_BROWSER_URL})"))

Running Neo4j loader...
Executing Neo4j loader command: python /home/radhikesh/Documents/chubb/angular-project-2/python_scripts/neo4j_loader.py --file /home/radhikesh/Documents/chubb/angular-project-2/output/parsed_angular_data.json --clear
Neo4j loader finished.
--- Loader STDOUT ---

--- Loader STDERR ---
2025-06-06 20:30:47,783 - INFO - Successfully connected to Neo4j at bolt://localhost:7687
2025-06-06 20:30:48,137 - INFO - Ensured AngularEntity.id uniqueness constraint.
2025-06-06 20:30:48,173 - INFO - Received notification from DBMS server: {severity: INFORMATION} {code: Neo.ClientNotification.Schema.IndexOrConstraintAlreadyExists} {category: SCHEMA} {title: `CREATE CONSTRAINT IF NOT EXISTS FOR (e:AngularEntity) REQUIRE (e.id) IS UNIQUE` has no effect.} {description: `CONSTRAINT constraint_b6c067e2 FOR (e:AngularEntity) REQUIRE (e.id) IS UNIQUE` already exists.} {position: None} for query: 'CREATE CONSTRAINT IF NOT EXISTS FOR (n:AngularEntity) REQUIRE n.id IS UNIQUE'
2025-06-06 2

<font color='green'>**Neo4j data loading completed.**</font> You can inspect the graph at [http://localhost:7474/browser/](http://localhost:7474/browser/)

## 3. Export Graph Structure and Data Model

Run the Python script to export `architecture.json`, `architecture_schema.json`, `data_model.json`, and `data_model_schema.json`.

In [6]:
def run_graph_exporter(python_scripts_path):
    print("Running graph exporter...")
    exporter_script = os.path.join(python_scripts_path, "graph_exporter.py")
    command = ["python", exporter_script]
    
    try:
        print(f"Executing graph exporter command: {' '.join(command)}")
        result = subprocess.run(command, check=True, capture_output=True, text=True, cwd=BASE_DIR)
        print("Graph exporter finished.")
        print("--- Exporter STDOUT ---")
        print(result.stdout)
        if result.stderr:
            print("--- Exporter STDERR ---")
            print(result.stderr)
        
        # Verify file creation
        expected_files = [
            os.path.join(OUTPUT_DIR, "architecture.json"),
            os.path.join(OUTPUT_DIR, "architecture_schema.json"),
            os.path.join(OUTPUT_DIR, "data_model.json"),
            os.path.join(OUTPUT_DIR, "data_model_schema.json")
        ]
        all_files_exist = True
        for f_path in expected_files:
            if not os.path.exists(f_path):
                print(f"Error: Expected file {f_path} was not created.")
                all_files_exist = False
        if all_files_exist:
            print("All expected JSON export files were created successfully.")
            return True
        else:
            return False
            
    except subprocess.CalledProcessError as e:
        print(f"Graph exporter script failed: {e}")
        print(f"Stdout:\n{e.stdout}")
        print(f"Stderr:\n{e.stderr}")
        return False
    except FileNotFoundError:
        print(f"Error: Python interpreter or script {exporter_script} not found.")
        return False

if not run_graph_exporter(PYTHON_SCRIPTS_DIR):
    display(Markdown("<font color='red'>**Graph exporting failed. Check logs above.**</font>"))
else:
    display(Markdown("<font color='green'>**Graph exporting completed successfully.**</font> Check the `output` directory."))

Running graph exporter...
Executing graph exporter command: python /home/radhikesh/Documents/chubb/angular-project-2/python_scripts/graph_exporter.py
Graph exporter finished.
--- Exporter STDOUT ---

--- Exporter STDERR ---
2025-06-06 20:30:52,173 - INFO - graph_exporter.py:25 - Output directory configured to: /home/radhikesh/Documents/chubb/angular-project-2/output
2025-06-06 20:30:52,173 - INFO - graph_exporter.py:44 - Output directory /home/radhikesh/Documents/chubb/angular-project-2/output already exists.
2025-06-06 20:30:52,173 - INFO - graph_exporter.py:591 - graph_exporter.py script started.
2025-06-06 20:30:52,173 - INFO - graph_exporter.py:600 - Successfully wrote startup test file to /home/radhikesh/Documents/chubb/angular-project-2/output/startup_test_output.txt
2025-06-06 20:30:52,173 - INFO - graph_exporter.py:602 - Removed startup test file /home/radhikesh/Documents/chubb/angular-project-2/output/startup_test_output.txt
2025-06-06 20:30:52,173 - INFO - graph_exporter.py:5

<font color='green'>**Graph exporting completed successfully.**</font> Check the `output` directory.

## 4. Graph Visualization and Sample Queries (using py2neo or neo4j driver)

This section demonstrates how to connect to Neo4j from Python within the notebook and run some queries. For rich, interactive graph visualization, the Neo4j Browser is often preferred. We can also use libraries like `pyvis` or `igraph` for basic plotting here.

In [7]:
from neo4j import GraphDatabase

NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

if not NEO4J_PASSWORD:
    print("NEO4J_PASSWORD not set in .env file. Skipping direct Neo4j connection.")
else:
    try:
        driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD), encrypted=False) # Adjust encrypted=True for SSL
        
        def run_query(tx, query, params=None):
            result = tx.run(query, params)
            return [record for record in result]

        with driver.session() as session:
            print("Connected to Neo4j for querying.")
            
            # Example Query 1: Count node types
            query1 = "MATCH (n:AngularEntity) RETURN labels(n) AS NodeType, count(*) AS Count ORDER BY Count DESC"
            results1 = session.execute_read(run_query, query1)
            df1 = pd.DataFrame([dict(record) for record in results1])
            display(Markdown("### Node Types and Counts:"))
            display(df1)

            # Example Query 2: Components and their injected services
            query2 = """
            MATCH (c:Component)-[:INJECTS]->(s:Service)
            RETURN c.name AS Component, s.name AS InjectedService, s.filePath AS ServiceFile
            LIMIT 10
            """
            results2 = session.execute_read(run_query, query2)
            df2 = pd.DataFrame([dict(record) for record in results2])
            display(Markdown("### Components and Injected Services (Sample):"))
            display(df2)
            
            # Example Query 3: Modules and their declarations
            query3 = """
            MATCH (m:Module)-[:DECLARES]->(d)
            RETURN m.name AS Module, d.name AS DeclaredEntity, labels(d) as EntityType
            LIMIT 10
            """
            results3 = session.execute_read(run_query, query3)
            df3 = pd.DataFrame([dict(record) for record in results3])
            display(Markdown("### Modules and Declared Entities (Sample):"))
            display(df3)

    except Exception as e:
        print(f"Error connecting to or querying Neo4j: {e}")
        display(Markdown(f"<font color='orange'>Could not connect to Neo4j directly from notebook: {e}. You can still use Neo4j Browser.</font>"))
    finally:
        if 'driver' in locals() and driver:
            driver.close()

Connected to Neo4j for querying.


### Node Types and Counts:

,NodeType,Count
0,"[File, AngularEntity]",40
1,"[Component, AngularEntity]",18
2,"[AngularEntity, Interface]",9
3,"[Service, AngularEntity]",6
4,"[AngularEntity, Directive]",1
5,"[AngularEntity, Pipe]",1


### Components and Injected Services (Sample):

,Component,InjectedService,ServiceFile
0,ArticleComponent,UserService,src/app/core/auth/services/user.service.ts
1,EditorComponent,UserService,src/app/core/auth/services/user.service.ts
2,ProfileComponent,UserService,src/app/core/auth/services/user.service.ts
3,FollowButtonComponent,UserService,src/app/core/auth/services/user.service.ts
4,SettingsComponent,UserService,src/app/core/auth/services/user.service.ts
5,AuthComponent,UserService,src/app/core/auth/services/user.service.ts
6,HomeComponent,UserService,src/app/core/auth/services/user.service.ts
7,FavoriteButtonComponent,UserService,src/app/core/auth/services/user.service.ts
8,ArticleComponent,ArticlesService,src/app/features/article/services/articles.ser...
9,EditorComponent,ArticlesService,src/app/features/article/services/articles.ser...


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: DECLARES)} {position: line: 2, column: 32, offset: 32} for query: '\n            MATCH (m:Module)-[:DECLARES]->(d)\n            RETURN m.name AS Module, d.name AS DeclaredEntity, labels(d) as EntityType\n            LIMIT 10\n            '


### Modules and Declared Entities (Sample):

""


### Basic Graph Visualization (using pyvis)

In [8]:
try:
    from pyvis.network import Network
    PYVIS_AVAILABLE = True
except ImportError:
    PYVIS_AVAILABLE = False
    print("pyvis library not found. Skipping pyvis visualization. To install: pip install pyvis")

if PYVIS_AVAILABLE and NEO4J_PASSWORD:
    try:
        driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD), encrypted=False)
        with driver.session() as session:
            # Fetch a small subgraph for visualization
            # Example: A specific component, its injections, and the module it's declared in
            # Choose a component name that exists in your graph
            target_component_name = "ArticleListComponent" # Change if needed
            
            query_subgraph = f"""
            MATCH (c:Component {{name: '{target_component_name}'}})
            OPTIONAL MATCH (c)-[r1:INJECTS]->(s:Service)
            OPTIONAL MATCH (m:Module)-[r2:DECLARES]->(c)
            OPTIONAL MATCH (c)-[r3:DEFINED_IN]->(f1:File)
            OPTIONAL MATCH (s)-[r4:DEFINED_IN]->(f2:File)
            OPTIONAL MATCH (m)-[r5:DEFINED_IN]->(f3:File)
            RETURN c, r1, s, m, r2, r3, f1, r4, f2, r5, f3
            """
            results_subgraph = session.run(query_subgraph)
            
            net = Network(notebook=True, height="750px", width="100%", cdn_resources='remote') # Use remote for offline friendly
            
            added_nodes = set()
            for record in results_subgraph:
                for key, value in record.items():
                    if value is None: continue
                    
                    if hasattr(value, 'labels'): # It's a node
                        node_id = value.element_id
                        if node_id not in added_nodes:
                            label = value.get('name', value.get('filePath', node_id))
                            title = json.dumps(dict(value.items()), indent=2)
                            node_type = list(value.labels - {'AngularEntity', '_Node'})[0] if value.labels - {'AngularEntity', '_Node'} else list(value.labels)[0]
                            
                            color_map = {
                                'Component': 'blue',
                                'Service': 'green',
                                'Module': 'red',
                                'File': 'grey',
                                'ExternalOrUnresolved': 'orange'
                            }
                            net.add_node(node_id, label=label, title=title, group=node_type, color=color_map.get(node_type, 'purple'))
                            added_nodes.add(node_id)
                            
                    elif hasattr(value, 'type'): # It's a relationship
                        net.add_edge(value.start_node.element_id, value.end_node.element_id, title=value.type, label=value.type)
            
            if not added_nodes:
                print(f"No data found for component '{target_component_name}' to visualize. Try a different component or check graph.")
            else:
                net.show_buttons(filter_=['physics'])
                # net.show("angular_graph_sample.html")
                # display(HTML("angular_graph_sample.html")) # if not using notebook=True directly
                display(net.show("angular_graph_sample.html")) # For direct rendering in output cell
                display(Markdown("Generated `angular_graph_sample.html`. If not displayed above, open it manually."))

    except Exception as e:
        print(f"Error during pyvis visualization: {e}")
    finally:
        if 'driver' in locals() and driver:
            driver.close()
else:
    if not PYVIS_AVAILABLE:
        print("Skipping pyvis visualization as pyvis is not available.")
    if not NEO4J_PASSWORD:
        print("Skipping pyvis visualization as Neo4j password is not available.")

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: DECLARES)} {position: line: 4, column: 43, offset: 163} for query: "\n            MATCH (c:Component {name: 'ArticleListComponent'})\n            OPTIONAL MATCH (c)-[r1:INJECTS]->(s:Service)\n            OPTIONAL MATCH (m:Module)-[r2:DECLARES]->(c)\n            OPTIONAL MATCH (c)-[r3:DEFINED_IN]->(f1:File)\n            OPTIONAL MATCH (s)-[r4:DEFINED_IN]->(f2:File)\n            OPTIONAL MATCH (m)-[r5:DEFINED_IN]->(f3:File)\n            RETURN c, r1, s, m, r2, r3, f1, r4, f2, r5, f3\n            "


Error during pyvis visualization: non existent node '4:95f7362c-4d9f-4d67-9dc0-c8fffb720ba4:138'


In [15]:
from langchain_community.graphs import Neo4jGraph
from langchain_community.llms import Ollama
from langchain.chains import GraphCypherQAChain # For Cypher generation and QA
from langchain_core.prompts import PromptTemplate
import logging
import os

# Configure LangChain logging for more verbosity if needed
# logging.basicConfig(level=logging.INFO) # General INFO for LangChain
# logging.getLogger("langchain.chains.graph_qa.base").setLevel(logging.DEBUG) # DEBUG for GraphCypherQAChain

OLLAMA_MODEL = "llama3:8b" # Or "mistral", "codellama:7b", etc. - make sure it's pulled in Ollama
OLLAMA_BASE_URL = "http://localhost:11434"

NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD", "test1234")
LANGCHAIN_INTEGRATION_POSSIBLE = True

if not NEO4J_PASSWORD:
    print("NEO4J_PASSWORD not set. LangChain integration with Neo4j will be skipped.")
    LANGCHAIN_INTEGRATION_POSSIBLE = False

graph = None # For Neo4jGraph instance
llm = None   # For Ollama LLM instance

if LANGCHAIN_INTEGRATION_POSSIBLE:
    try:
        # Initialize Neo4jGraph connection (used by LangChain)
        graph = Neo4jGraph(
            url=NEO4J_URI, # Defined in an earlier cell
            username=NEO4J_USER, # Defined in an earlier cell
            password=NEO4J_PASSWORD
        )
        print(f"Neo4jGraph initialized for LangChain.")
        
        # It's good practice to see what schema LangChain can infer, though we'll provide a manual one too.
        # print("Attempting to refresh and display Neo4jGraph schema (can be verbose)...")
        # graph.refresh_schema() 
        # print(graph.schema) # This prints the schema LangChain sees

        # For a quicker check:
        print("Neo4jGraph connection test (labels and rel types):")
        print(f"  Node Labels from DB: {graph.query('CALL db.labels() YIELD label RETURN label')[:5]}") # Show first 5
        print(f"  Relationship Types from DB: {graph.query('CALL db.relationshipTypes() YIELD relationshipType RETURN relationshipType')[:5]}")

    except Exception as e:
        print(f"Error initializing Neo4jGraph for LangChain: {e}")
        LANGCHAIN_INTEGRATION_POSSIBLE = False

    if LANGCHAIN_INTEGRATION_POSSIBLE: # Proceed only if graph connection was successful
        try:
            # Initialize Ollama LLM
            llm = Ollama(base_url=OLLAMA_BASE_URL, model=OLLAMA_MODEL, temperature=0)
            print(f"Ollama LLM initialized with model: {OLLAMA_MODEL}")
            
            # Test LLM connection with a simple query
            print("Testing Ollama LLM connection...")
            test_response = llm.invoke("Briefly, what is Neo4j?")
            print(f"LLM Test Response (first 100 chars): {test_response}...")

        except Exception as e:
            print(f"Error initializing Ollama LLM: {e}")
            print(f"Please ensure Ollama is running (e.g., 'ollama serve') and the model '{OLLAMA_MODEL}' is pulled (e.g., 'ollama pull {OLLAMA_MODEL}').")
            LANGCHAIN_INTEGRATION_POSSIBLE = False

if not LANGCHAIN_INTEGRATION_POSSIBLE:
    display(Markdown("<font color='orange'>**Skipping LangChain SLM integration due to missing Neo4j password or Ollama/Neo4j connection issues.**</font>"))
else:
    display(Markdown("<font color='green'>**LangChain components (Neo4jGraph, Ollama LLM) initialized successfully.**</font>"))

Neo4jGraph initialized for LangChain.
Neo4jGraph connection test (labels and rel types):
  Node Labels from DB: [{'label': 'BaseNode'}, {'label': 'File'}, {'label': 'Component'}, {'label': 'Service'}, {'label': 'Module'}]
  Relationship Types from DB: [{'relationshipType': 'IMPORTS'}, {'relationshipType': 'DEFINED_IN'}, {'relationshipType': 'INJECTS'}, {'relationshipType': 'IMPLEMENTS'}]
Ollama LLM initialized with model: llama3:8b
Testing Ollama LLM connection...
LLM Test Response (first 100 chars): Neo4j is a graph database management system that allows you to store and query data as nodes (vertices) connected by relationships. It's designed to handle complex networks and relationships between entities, making it particularly useful for applications that require querying and analyzing interconnected data.

In traditional relational databases, data is stored in tables with well-defined schemas, whereas Neo4j stores data as a graph of nodes and edges. This allows for efficient querying

<font color='green'>**LangChain components (Neo4jGraph, Ollama LLM) initialized successfully.**</font>

### 5.1 Using `GraphCypherQAChain` for Question Answering

`GraphCypherQAChain` attempts to:
1. Understand the user's question.
2. Inspect the graph schema (or use a provided schema context).
3. Generate a Cypher query to retrieve relevant information from the Neo4j graph.
4. Execute the Cypher query against the graph.
5. Pass the query results and the original question to the LLM to synthesize a final, human-readable answer.

In [13]:
if LANGCHAIN_INTEGRATION_POSSIBLE and graph is not None and llm is not None:
    # --- Define a good schema description for the LLM ---
    try:
        raw_node_labels = graph.query("CALL db.labels() YIELD label RETURN label")
        node_labels_list = sorted([item['label'] for item in raw_node_labels if item['label'] not in ['_Node', 'AngularEntity', 'BaseNode']]) # Exclude BaseNode too if it's generic
        node_labels_str = ", ".join(node_labels_list)

        raw_rel_types = graph.query("CALL db.relationshipTypes() YIELD relationshipType RETURN relationshipType")
        rel_types_list = sorted([item['relationshipType'] for item in raw_rel_types])
        rel_types_str = ", ".join(rel_types_list)
        
        print(f"Effective Node Labels for LLM Prompt: {node_labels_str}")
        print(f"Effective Relationship Types for LLM Prompt: {rel_types_str}")

    except Exception as e:
        print(f"Could not dynamically fetch schema components for LLM prompt: {e}")
        node_labels_str = "Component, Directive, ExternalOrUnresolved, File, Interface, Module, Pipe, Service" # Fallback
        rel_types_str = "DEFINED_IN, IMPLEMENTS, IMPORTS, INJECTS, DECLARES, IMPORTS_MODULE, PROVIDES" # Add missing ones if needed from your actual graph
    
    # This is the text description of your schema you provide to the LLM.
    MANUAL_GRAPH_SCHEMA_DESCRIPTION = f"""
The graph database contains information about an Angular codebase.
Node Labels represent different Angular entities: {node_labels_str}.
Relationship Types describe how these entities are connected: {rel_types_str}.

Key Node Properties:
- All nodes typically have an 'id' (string, unique business identifier, e.g., "Component:AppComponent:src/app/app.component.ts") and a 'name' (string, e.g., "AppComponent").
- 'File' nodes have 'filePath' (string, relative path).
- 'Component' nodes can have 'selector' (string, e.g., "app-root"), 'templateUrl' (string), 'styleUrls' (list of strings).
- 'Service' nodes can have 'providedIn' (string, e.g., "root").
- 'Module' nodes connect to other entities via relationships.
- 'ExternalOrUnresolved' nodes represent external libraries or unresolved dependencies, with a 'name' property (e.g., "rxjs", "Unresolved:SomeType") and an 'originalId' (e.g., "External:rxjs").

Key Relationship Meanings (ensure these match your actual graph from the parser output):
- (entity)-[:DEFINED_IN]->(file:File) : An entity (like Component, Service) is defined in a specific file.
- (module:Module)-[:DECLARES]->(declarable:Component|Pipe|Directive) : A module declares a component, pipe, or directive.
- (module:Module)-[:IMPORTS_MODULE]->(imported_module:Module) : A module imports another module.
- (module:Module)-[:PROVIDES]->(service:Service) : A module makes a service available for injection.
- (injector:Component|Service)-[:INJECTS]->(injectee:Service) : A component or service injects another service. The relationship might have a 'parameterName' property.
- (file:File)-[:IMPORTS]->(target:File|ExternalOrUnresolved) : A file imports code from another file or an external library. The relationship has a 'from' property (string, the original import path).
- (class)-[:IMPLEMENTS]->(interface:Interface) : A class implements an interface.

When querying, always use the specific labels like 'Component' or 'Service'.
String values in Cypher queries (like names or filePaths) MUST be enclosed in single or double quotes.
For example, to find a component named 'AppComponent', use: MATCH (c:Component {{name: 'AppComponent'}}) RETURN c.name, c.filePath
"""

    # --- Construct a custom Cypher generation prompt ---
    # This template is based on the one LangChain uses internally but allows us to inject our detailed schema.
    CYPHER_GENERATION_PROMPT_TEMPLATE = """Task:Generate Cypher query to query a graph database.
Instructions:
Use only the node labels and relationship types provided in the schema.
Do not use any other node labels or relationship types.
The Cypher query must be syntactically correct and be supported by Neo4j.
Always quote string values in the Cypher query. For example, use `name: 'MyComponent'` not `name: MyComponent`.
Only return the Cypher query, no other text or explanation.

Schema:
{schema}

Question: {question}
Cypher Query:"""

    CYPHER_PROMPT = PromptTemplate(
        input_variables=["schema", "question"],
        template=CYPHER_GENERATION_PROMPT_TEMPLATE,
    )

    # --- Initialize GraphCypherQAChain with the custom prompt ---
    try:
        chain = GraphCypherQAChain.from_llm(
            llm=llm,
            graph=graph,
            cypher_prompt=CYPHER_PROMPT.partial(schema=MANUAL_GRAPH_SCHEMA_DESCRIPTION), # Inject our schema description into the prompt
            verbose=True, 
            allow_dangerous_requests=True
            # validate_cypher=True, # Keep commented for now unless debugging specific Cypher issues
            # top_k=5 
        )
        print("GraphCypherQAChain initialized successfully with custom prompt and schema.")
        
        # --- Define questions for the RAG system ---
        questions_to_ask = [
            "What services does the ArticleModule provide?",
            "Show me components declared in the SharedModule.",
            "Which services are injected into ArticleListComponent?", 
            "Find the selector for the component named FollowButtonComponent.",
            "List up to 3 files that import the ApiService." 
        ]

        for i, question_text in enumerate(questions_to_ask):
            display(Markdown(f"---"))
            display(Markdown(f"### RAG Question {i + 1}: {question_text}"))
            try:
                response = chain.invoke({"query": question_text})
                display(Markdown(f"**LLM Answer:** {response['result']}"))
            except Exception as e:
                print(f"Error running LangChain RAG chain for question '{question_text}':")
                print(f"  Type: {type(e)}")
                print(f"  Error: {e}")
                display(Markdown(f"<font color='red'>**Error processing question:** {e}. Check console for details.</font>"))

    except Exception as e:
        print(f"An error occurred during LangChain RAG setup or execution: {e}")
        display(Markdown(f"<font color='red'>**LangChain RAG processing failed:** {e}</font>"))
else:
    display(Markdown("<font color='orange'>**Skipping LangChain GraphCypherQAChain example as prerequisites (Neo4j connection, Ollama LLM) are not met.**</font>"))

Effective Node Labels for LLM Prompt: Component, Directive, ExternalOrUnresolved, File, Interface, Module, Pipe, Service
Effective Relationship Types for LLM Prompt: DEFINED_IN, IMPLEMENTS, IMPORTS, INJECTS
GraphCypherQAChain initialized successfully with custom prompt and schema.


---

### RAG Question 1: What services does the ArticleModule provide?



> Entering new GraphCypherQAChain chain...


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: PROVIDES)} {position: line: 1, column: 44, offset: 43} for query: "MATCH (m:Module {name: 'ArticleModule'})-[:PROVIDES]->(s:Service) RETURN s.name, s.id"


...

**LLM Answer:** I don't know the answer.

---

### RAG Question 5: List up to 3 files that import the ApiService.



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (f:File)-[:IMPORTS]->(target) WHERE target.name = 'ApiService' RETURN f.filePath AS filePath;
Full Context:
[]

> Finished chain.


**LLM Answer:** I don't know the answer.

In [17]:
if LANGCHAIN_INTEGRATION_POSSIBLE and graph is not None and llm is not None:
    # --- Define an accurate schema description for the LLM ---
    # This description is critical for the LLM to generate correct Cypher queries.
    
    # Dynamically fetch labels and relationship types from your actual graph
    # to ensure the description is as accurate as possible.
    try:
        raw_node_labels_result = graph.query("CALL db.labels() YIELD label RETURN label")
        # Filter out generic/internal labels that might confuse the LLM
        node_labels_list = sorted([
            item['label'] for item in raw_node_labels_result 
            if item['label'] not in ['_Node', 'AngularEntity', 'BaseNode', 'UnknownEntity', 'ExternalOrUnresolved', 'UnknownNode'] # Add any other generic labels to exclude
        ])
        # Add back 'ExternalOrUnresolved' explicitly as it's a special type we handle
        if 'ExternalOrUnresolved' not in node_labels_list and graph.query("MATCH (n:ExternalOrUnresolved) RETURN count(n) > 0 AS ex LIMIT 1")[0]['ex']:
            node_labels_list.append('ExternalOrUnresolved')
        node_labels_str = ", ".join(node_labels_list)


        raw_rel_types_result = graph.query("CALL db.relationshipTypes() YIELD relationshipType RETURN relationshipType")
        rel_types_list = sorted([item['relationshipType'] for item in raw_rel_types_result])
        rel_types_str = ", ".join(rel_types_list)
        
        print(f"EFFECTIVE NODE LABELS FOR LLM PROMPT: {node_labels_str}")
        print(f"EFFECTIVE RELATIONSHIP TYPES FOR LLM PROMPT: {rel_types_str}")

    except Exception as e:
        print(f"Warning: Could not dynamically fetch schema components for LLM prompt: {e}")
        # Fallback if DB query fails - **ADJUST THESE TO YOUR EXPECTED PARSED LABELS/RELS**
        node_labels_str = "Class, Component, Directive, File, Interface, Module, Pipe, Service, ExternalOrUnresolved"
        rel_types_str = "BOOTSTRAPS, DECLARES, DEFINED_IN, EXPORTS_MODULE, IMPLEMENTS, IMPORTS, IMPORTS_MODULE, INJECTS, PROVIDES"
        print(f"Using FALLBACK NODE LABELS: {node_labels_str}")
        print(f"Using FALLBACK RELATIONSHIP TYPES: {rel_types_str}")
    
    # This is the text description of your schema you provide to the LLM.
    # Be very specific about node properties it should use.
    MANUAL_GRAPH_SCHEMA_DESCRIPTION = f"""
The graph database contains information about an Angular codebase.
Relevant Node Labels are: {node_labels_str}.
Relevant Relationship Types are: {rel_types_str}.

Key Node Properties and Details:
- Common properties for most entities: 'id' (string, unique business identifier like "Component:Name:path/to/file.ts"), 'name' (string, e.g., "AppComponent"), 'filePath' (string, relative path to the file where it's defined).
- 'Component': Can have 'selector' (string), 'templateUrl' (string), 'styleUrls' (list of strings).
- 'Service': Can have 'providedIn' (string, e.g., "root").
- 'Module': Represents an NgModule.
- 'File': Represents a .ts source file.
- 'Pipe': Represents an Angular Pipe.
- 'Directive': Represents an Angular Directive.
- 'Interface': Represents a TypeScript interface.
- 'Class': Represents a generic TypeScript class if not a more specific Angular entity.
- 'ExternalOrUnresolved': Represents external libraries (e.g., 'name': "rxjs") or internal symbols that could not be fully resolved by the parser (e.g., 'name': "Unresolved:SomeType"). It has an 'originalId' property reflecting the parser's placeholder.

Key Relationship Meanings and Usage:
- (entity)-[:DEFINED_IN]->(file:File): Indicates that 'entity' (e.g., a Component, Service, Module) is defined in the specified 'file'.
- (module:Module)-[:DECLARES]->(item:Component|Pipe|Directive): The 'module' declares 'item'. The 'item' node will have a specific label like :Component, :Pipe, or :Directive.
- (module:Module)-[:PROVIDES]->(item:Service): The 'module' provides 'item', which is typically a Service. The 'item' node will usually have the :Service label.
- (module:Module)-[:IMPORTS_MODULE]->(importedModule:Module): The 'module' imports another 'importedModule'. Both nodes will have the :Module label.
- (module:Module)-[:EXPORTS_MODULE]->(exportedItem): The 'module' exports 'exportedItem'. The 'exportedItem' could be another Module, or a Component, Pipe, or Directive that was declared or imported by this module. The label of 'exportedItem' will be specific (e.g., :Module, :Component).
- (module:Module)-[:BOOTSTRAPS]->(component:Component): The 'module' bootstraps a root 'component'. The 'component' node will have the :Component label.
- (source:Component|Service)-[:INJECTS]->(target:Service): The 'source' (a Component or another Service) injects the 'target' Service. This relationship may have a 'parameterName' property.
- (file:File)-[:IMPORTS]->(target): The 'file' imports from 'target'. 'target' can be another 'File' node within the project or an 'ExternalOrUnresolved' node (for libraries like 'rxjs' or unparsed local files). This relationship has a 'from' property (string, the original import path string from the code).
- (class:Class|Component|Service|Directive)-[:IMPLEMENTS]->(interface:Interface): The 'class' (or specific entity type that is a class) implements the 'interface'.

Querying Guidelines for LLM:
- Always use the specific node labels mentioned (e.g., :Component, :Service, :Module) in your MATCH patterns.
- String values in Cypher MATCH patterns (e.g., for properties like 'name' or 'filePath') MUST be enclosed in single or double quotes (e.g., `{{name: 'ArticleModule'}}`).
- When asked about what a module "provides" or "declares", use the :PROVIDES or :DECLARES relationships respectively, originating from a :Module node.
- For dependencies, :INJECTS shows service injection, and :IMPORTS (on :File nodes) shows file-level imports.
"""

    # --- Construct a custom Cypher generation prompt ---
    CYPHER_GENERATION_PROMPT_TEMPLATE = """Task:Generate Cypher query to query a graph database.
Instructions:
Use only the node labels and relationship types provided in the schema. Do not use any other node labels or relationship types not listed.
The Cypher query must be syntactically correct and be supported by Neo4j.
Always quote string values in the Cypher query. For example, use `name: 'MyComponent'` not `name: MyComponent`.
Only return the Cypher query itself, with no additional text, explanations, or markdown formatting.

Schema:
{schema}

Question: {question}
Cypher Query:""" # Ensure no trailing newlines or spaces after "Cypher Query:" that might confuse the LLM

    CYPHER_PROMPT = PromptTemplate(
        input_variables=["schema", "question"],
        template=CYPHER_GENERATION_PROMPT_TEMPLATE,
    )

    # --- Initialize GraphCypherQAChain with the custom prompt ---
    try:
        chain = GraphCypherQAChain.from_llm(
            llm=llm,
            graph=graph,
            cypher_prompt=CYPHER_PROMPT.partial(schema=MANUAL_GRAPH_SCHEMA_DESCRIPTION), # Inject our schema description
            verbose=True, 
            allow_dangerous_requests =True, # Allow LLM to generate Cypher queries that might be dangerous (e.g., DELETE)
            # validate_cypher=True, # Optional: set to True for debugging LLM's Cypher. Can be slow.
            # top_k=5 # Optional: limit number of graph results passed to LLM for final answer
        )
        print("GraphCypherQAChain initialized successfully with custom prompt and schema.")
        
        # --- Define questions for the RAG system ---
        questions_to_ask = [
            "What services does the ArticleModule provide?", # Should use :PROVIDES
            "Show me components declared in the SharedModule.", # Should use :DECLARES
            "Which services are injected into ArticleListComponent?", # Should use :INJECTS
            "Find the selector for the component named FollowButtonComponent.", # Needs to find Component node and its 'selector' property
            "List up to 3 files that import the ApiService.", # This is more complex: find ApiService, then its File via DEFINED_IN, then Files that IMPORT that File.
            "Which modules import the SharedModule?", # Should use :IMPORTS_MODULE
            "What components are bootstrapped by the AppModule?" # Should use :BOOTSTRAPS
        ]

        for i, question_text in enumerate(questions_to_ask):
            display(Markdown(f"---"))
            display(Markdown(f"### RAG Question {i + 1}: {question_text}"))
            try:
                response = chain.invoke({"query": question_text})
                display(Markdown(f"**LLM Answer:** {response['result']}"))
            except Exception as e:
                print(f"Error running LangChain RAG chain for question '{question_text}':")
                print(f"  Type: {type(e)}")
                print(f"  Error: {e}")
                # The `verbose=True` in the chain should print the generated Cypher before an execution error.
                display(Markdown(f"<font color='red'>**Error processing question:** {e}. Check console for details, including any generated Cypher if verbose=True.</font>"))

    except Exception as e:
        print(f"An error occurred during LangChain RAG setup or execution: {e}")
        display(Markdown(f"<font color='red'>**LangChain RAG processing failed:** {e}</font>"))
else:
    display(Markdown("<font color='orange'>**Skipping LangChain GraphCypherQAChain example as prerequisites (Neo4j connection, Ollama LLM) are not met.**</font>"))

EFFECTIVE NODE LABELS FOR LLM PROMPT: Component, Directive, File, Interface, Module, Pipe, Service, ExternalOrUnresolved
EFFECTIVE RELATIONSHIP TYPES FOR LLM PROMPT: DEFINED_IN, IMPLEMENTS, IMPORTS, INJECTS
GraphCypherQAChain initialized successfully with custom prompt and schema.


---

### RAG Question 1: What services does the ArticleModule provide?



> Entering new GraphCypherQAChain chain...


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: PROVIDES)} {position: line: 1, column: 49, offset: 48} for query: "MATCH (module:Module {name: 'ArticleModule'})-[:PROVIDES]->(service:Service) RETURN service;"


...

### RAG Question 7: What components are bootstrapped by the AppModule?



> Entering new GraphCypherQAChain chain...


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.UnknownRelationshipTypeWarning} {category: UNRECOGNIZED} {title: The provided relationship type is not in the database.} {description: One of the relationship types in your query is not available in the database, make sure you didn't misspell it or that the label is available when you run this statement in your application (the missing relationship type is: BOOTSTRAPS)} {position: line: 1, column: 45, offset: 44} for query: "MATCH (module:Module {name: 'AppModule'})-[:BOOTSTRAPS]->(component:Component) RETURN component;"


Generated Cypher:
MATCH (module:Module {name: 'AppModule'})-[:BOOTSTRAPS]->(component:Component) RETURN component;
Full Context:
[]

> Finished chain.


**LLM Answer:** I don't know the answer.

In [19]:
if LANGCHAIN_INTEGRATION_POSSIBLE and graph is not None and llm is not None:
    # --- Define an accurate schema description for the LLM ---
    # This description is critical for the LLM to generate correct Cypher queries.

    # Dynamically fetch labels and relationship types from your actual graph
    # to ensure the description is as accurate as possible.
    try:
        raw_node_labels_result = graph.query(
            "CALL db.labels() YIELD label RETURN label"
        )
        # Filter out generic/internal labels that might confuse the LLM
        node_labels_list = sorted(
            [
                item["label"]
                for item in raw_node_labels_result
                if item["label"]
                not in [
                    "_Node",
                    "AngularEntity",
                    "BaseNode",
                    "UnknownEntity",
                    "ExternalOrUnresolved",
                    "UnknownNode",
                ]  # Add any other generic labels to exclude
            ]
        )
        # Add back 'ExternalOrUnresolved' explicitly as it's a special type we handle
        if (
            "ExternalOrUnresolved" not in node_labels_list
            and graph.query(
                "MATCH (n:ExternalOrUnresolved) RETURN count(n) > 0 AS ex LIMIT 1"
            )[0]["ex"]
        ):
            node_labels_list.append("ExternalOrUnresolved")
        node_labels_str = ", ".join(node_labels_list)

        raw_rel_types_result = graph.query(
            "CALL db.relationshipTypes() YIELD relationshipType RETURN relationshipType"
        )
        rel_types_list = sorted(
            [item["relationshipType"] for item in raw_rel_types_result]
        )
        rel_types_str = ", ".join(rel_types_list)

        print(f"EFFECTIVE NODE LABELS FOR LLM PROMPT: {node_labels_str}")
        print(f"EFFECTIVE RELATIONSHIP TYPES FOR LLM PROMPT: {rel_types_str}")

    except Exception as e:
        print(
            f"Warning: Could not dynamically fetch schema components for LLM prompt: {e}"
        )
        # Fallback if DB query fails - **ADJUST THESE TO YOUR EXPECTED PARSED LABELS/RELS**
        node_labels_str = "Class, Component, Directive, File, Interface, Module, Pipe, Service, ExternalOrUnresolved"
        rel_types_str = "BOOTSTRAPS, DECLARES, DEFINED_IN, EXPORTS_MODULE, IMPLEMENTS, IMPORTS, IMPORTS_MODULE, INJECTS, PROVIDES"
        print(f"Using FALLBACK NODE LABELS: {node_labels_str}")
        print(f"Using FALLBACK RELATIONSHIP TYPES: {rel_types_str}")

    # This is the text description of your schema you provide to the LLM.
    # Be very specific about node properties it should use.
    MANUAL_GRAPH_SCHEMA_DESCRIPTION = f"""
The graph database contains information about an Angular codebase.
Relevant Node Labels found in the graph are: {node_labels_str}.
Relevant Relationship Types found in the graph are: {rel_types_str}. (Note: 'DEFINED_IN' relationships from entities to their defining File are also present but added during data loading).

Key Node Properties and Details:
- Common properties: 'id' (string, unique business identifier like "Component:Name:path/to/file.ts"), 'name' (string, e.g., "AppComponent"), 'filePath' (string, relative path to the file).
- 'File': Represents a .ts source file.
- 'Component': Has 'selector' (string), 'templateUrl' (string), 'styleUrls' (list of strings).
- 'Service': Has 'providedIn' (string, e.g., "root").
- 'Directive': Has 'selector' (string).
- 'Pipe': Has 'pipeName' (string).
- 'Interface': Represents a TypeScript interface.
- 'ExternalOrUnresolved': Node created by data loader for `targetId`s prefixed with `External:`, `Unresolved:`, or `Ambiguous:`. It has a 'name' property (e.g., "rxjs", "HttpClient", "OnInit") and an 'originalId' property.

Key Relationship Meanings and Usage (based on current parser output):
- (source_entity)-[:DEFINED_IN]->(file_node:File): Indicates that 'source_entity' (e.g., a Component, Service) is defined in the 'file_node'. (This relationship is added by the neo4j_loader.py script, not directly in the parser's JSON output).
- (source_file:File)-[:IMPORTS]->(target:ExternalOrUnresolved): The 'source_file' imports something. The 'target' node will be an 'ExternalOrUnresolved' node, with its 'name' property often reflecting the imported module path (e.g., "@angular/core", "./jwt.service") or type name. The relationship has a 'from' property (string, the original import path string from the code).
- (injector:Component|Service|Directive)-[:INJECTS]->(injectee): The 'injector' injects 'injectee'. 'injectee' can be a resolved 'Service' node OR an 'ExternalOrUnresolved' node (e.g., for 'HttpClient', 'Router'). The relationship has a 'parameterName' property.
- (class_entity:Component|Service|Directive|Pipe)-[:IMPLEMENTS]->(interface_target:ExternalOrUnresolved): The 'class_entity' implements an interface. 'interface_target' will be an 'ExternalOrUnresolved' node with its 'name' property being the interface name (e.g., "OnInit", "PipeTransform").

Querying Guidelines for LLM:
- Use the specific node labels: Component, Service, Directive, Pipe, File, Interface, ExternalOrUnresolved.
- String values in Cypher MATCH patterns (e.g., for properties like 'name' or 'filePath') MUST be enclosed in single or double quotes (e.g., `{{name: 'UserService'}}`).
- To find what a file imports, query `(f:File {{name: 'some.file.ts'}})-[:IMPORTS]->(ext:ExternalOrUnresolved) RETURN ext.name, ext.originalId`.
- To find what is injected into a component/service, query `(c:Component {{name: 'SomeComponent'}})-[:INJECTS]->(dep) RETURN dep.name, labels(dep)`. 'dep' might be a :Service or :ExternalOrUnresolved.
- Questions about Modules (declarations, providers, module imports) CANNOT be answered with direct relationships like :DECLARES or :PROVIDES with the current graph structure.
"""

    # --- Construct a custom Cypher generation prompt ---
    CYPHER_GENERATION_PROMPT_TEMPLATE = """Task:Generate Cypher query to query a graph database.
Instructions:
Use only the node labels and relationship types provided in the schema. Do not use any other node labels or relationship types not listed.
The Cypher query must be syntactically correct and be supported by Neo4j.
Always quote string values in the Cypher query. For example, use `name: 'MyComponent'` not `name: MyComponent`.
IMPORTANT: Your response MUST be ONLY the Cypher query itself. Do NOT include any other text, explanations, or markdown formatting like ```.
Start your response directly with 'MATCH', 'OPTIONAL MATCH', or 'RETURN' if it's a simple return.

Schema:
{schema}

Question: {question}
Cypher Query:"""  # Ensure no trailing newlines or spaces
    CYPHER_PROMPT = PromptTemplate(
        input_variables=["schema", "question"],
        template=CYPHER_GENERATION_PROMPT_TEMPLATE,
    )

    # --- Initialize GraphCypherQAChain with the custom prompt ---
    try:
        chain = GraphCypherQAChain.from_llm(
            llm=llm,
            graph=graph,
            cypher_prompt=CYPHER_PROMPT.partial(
                schema=MANUAL_GRAPH_SCHEMA_DESCRIPTION
            ),  # Inject our schema description
            verbose=True,
            allow_dangerous_requests=True,  # Allow LLM to generate Cypher queries that might be dangerous (e.g., DELETE)
            # validate_cypher=True, # Optional: set to True for debugging LLM's Cypher. Can be slow.
            # top_k=5 # Optional: limit number of graph results passed to LLM for final answer
        )
        print(
            "GraphCypherQAChain initialized successfully with custom prompt and schema."
        )

        # --- Define questions for the RAG system ---
        questions_to_ask = [
            # INJECTS relationship queries
            "What does the UserService inject?",
            "Which services or dependencies are injected into the IfAuthenticatedDirective?",
            "Find components or services that inject the JwtService.",  # Reverse query
            "What is the parameter name when ArticlesService injects HttpClient?",  # Tests rel property (if HttpClient resolves to ExternalOrUnresolved)
            # IMPORTS relationship queries (will point to ExternalOrUnresolved nodes)
            "What does the file 'jwt.service.ts' import?",  # Should be src/app/core/auth/services/jwt.service.ts
            "List imports for 'user.service.ts'.",
            "Which files import '@angular/core'?",  # This will be many files pointing to the same ExternalOrUnresolved node
            # IMPLEMENTS relationship queries (will point to ExternalOrUnresolved nodes)
            "What interfaces does the IfAuthenticatedDirective implement?",
            "Which directives or pipes implement 'OnInit'?",  # 'OnInit' will be an ExternalOrUnresolved node
            # DEFINED_IN relationship queries (this relies on neo4j_loader.py adding it)
            "In which file is the JwtService defined?",
            "What components or services are defined in the file 'if-authenticated.directive.ts'?",
            # Property-based queries
            "What is the selector for the HeaderComponent?",
            "Find services that are 'providedIn: root'.",
            "What is the pipeName for the MarkdownPipe?",
            # More complex, potentially multi-hop using existing relationships
            "Find services that are injected by components defined in 'article-list.component.ts'.",  # Component -> DEFINED_IN -> File, Component -> INJECTS -> Service
            "List the original import paths ('from' property) for all imports in 'user.service.ts'.",
        ]

        for i, question_text in enumerate(questions_to_ask):
            display(Markdown(f"---"))
            display(Markdown(f"### RAG Question {i + 1}: {question_text}"))
            try:
                response = chain.invoke({"query": question_text})
                display(Markdown(f"**LLM Answer:** {response['result']}"))
            except Exception as e:
                print(
                    f"Error running LangChain RAG chain for question '{question_text}':"
                )
                print(f"  Type: {type(e)}")
                print(f"  Error: {e}")
                # The `verbose=True` in the chain should print the generated Cypher before an execution error.
                display(
                    Markdown(
                        f"<font color='red'>**Error processing question:** {e}. Check console for details, including any generated Cypher if verbose=True.</font>"
                    )
                )

    except Exception as e:
        print(f"An error occurred during LangChain RAG setup or execution: {e}")
        display(
            Markdown(
                f"<font color='red'>**LangChain RAG processing failed:** {e}</font>"
            )
        )
else:
    display(
        Markdown(
            "<font color='orange'>**Skipping LangChain GraphCypherQAChain example as prerequisites (Neo4j connection, Ollama LLM) are not met.**</font>"
        )
    )

EFFECTIVE NODE LABELS FOR LLM PROMPT: Component, Directive, File, Interface, Module, Pipe, Service, ExternalOrUnresolved
EFFECTIVE RELATIONSHIP TYPES FOR LLM PROMPT: DEFINED_IN, IMPLEMENTS, IMPORTS, INJECTS
GraphCypherQAChain initialized successfully with custom prompt and schema.


---

### RAG Question 1: What does the UserService inject?



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (s:Service {name: 'UserService'})-[:INJECTS]->(dep) RETURN dep.name, labels(dep)
Full Context:
[{'dep.name': 'Router', 'labels(dep)': ['ExternalOrUnresolved']}, {'dep.name': 'JwtService', 'labels(dep)': ['Service', 'AngularEntity']}, {'dep.name': 'HttpClient', 'labels(dep)': ['ExternalOrUnresolved']}]

> Finished chain.


**LLM Answer:** Router, JwtService, and HttpClient.

...

**LLM Answer:** I don't know the answer.

---

### RAG Question 16: List the original import paths ('from' property) for all imports in 'user.service.ts'.



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (f:File {name: 'user.service.ts'})-[:IMPORTS]->(ext:ExternalOrUnresolved)
RETURN ext.name, f.from AS originalImportPath
Full Context:
[{'ext.name': '@angular/router', 'originalImportPath': None}, {'ext.name': '../user.model', 'originalImportPath': None}, {'ext.name': '@angular/common/http', 'originalImportPath': None}, {'ext.name': 'rxjs/operators', 'originalImportPath': None}, {'ext.name': './jwt.service', 'originalImportPath': None}, {'ext.name': 'rxjs', 'originalImportPath': None}, {'ext.name': '@angular/core', 'originalImportPath': None}]

> Finished chain.


**LLM Answer:** I don't know the answer.

In [ ]:
# In main.ipynb, NEW Cell (e.g., Cell D, after the previous Q&A loop)

from langchain_core.runnables import RunnablePassthrough # Usually for more complex LCEL chains
from langchain_core.output_parsers import StrOutputParser # For parsing LLM string output
from langchain.chains import LLMChain # ***** ADD THIS IMPORT *****

if LANGCHAIN_INTEGRATION_POSSIBLE and graph is not None and llm is not None and 'chain' in locals() and chain is not None:
    display(Markdown("## 5.2 Handling Complex Questions with Decomposition (Chain of Thought)\n"))

    # --- Define a Complex Question and its Decomposition ---
    # Original complex_question_1 was about HeaderComponent, which had no direct injections in your sample.
    # complex_question_1 = "Which services are ultimately used by the 'HeaderComponent', either directly injected or injected into services that the HeaderComponent itself injects (one level deep for service-to-service injection)?"
    
    # Revised complex question targeting IfAuthenticatedDirective which has known injections from your sample
    complex_question_2 = "What services are ultimately used by the 'IfAuthenticatedDirective', considering services it injects directly, and then services that THOSE directly injected services might inject (one further level deep)?"
    
    # --- Function to execute a single question and get context ---
    def get_context_for_question(q_text, current_qa_chain): # Renamed current_chain to current_qa_chain for clarity
        print(f"\n>>> Executing sub-question for context: \"{q_text}\"")
        try:
            response = current_qa_chain.invoke({"query": q_text})
            context = response.get('result', "No specific information found for this sub-question.")
            print(f"<<< Context from sub-question: \"{context}\"")
            return context
        except Exception as e:
            print(f"!!! Error executing sub-question '{q_text}': {e}")
            return f"Error obtaining information for sub-question: {q_text}."

    # --- Process Complex Question 2 ---
    display(Markdown(f"---"))
    display(Markdown(f"### Processing Complex Question: {complex_question_2}"))
    collected_contexts_for_synthesis = [] # Store contexts for the final synthesis

    # 1. Get services directly injected into IfAuthenticatedDirective
    sub_q_direct_injections_text = "Which services are directly injected into the IfAuthenticatedDirective?"
    context_direct_injections = get_context_for_question(sub_q_direct_injections_text, chain) # 'chain' is our GraphCypherQAChain
    
    if "Error obtaining information" not in context_direct_injections and "No specific information found" not in context_direct_injections:
        collected_contexts_for_synthesis.append(f"Information about direct injections for IfAuthenticatedDirective: {context_direct_injections}")

        # 2. Attempt to identify service names from the previous answer and find their dependencies.
        #    This requires some form of entity extraction. For this example, we'll make assumptions
        #    based on expected output or look for known service names.
        #    Your JSON sample showed IfAuthenticatedDirective injects UserService.
        
        # A very basic way to "extract" a service name (highly unreliable for general LLM output):
        # In a real system, use a dedicated LLM call for extraction or prompt the QA chain to list names clearly.
        directly_injected_service_names = []
        if "UserService" in context_direct_injections: # Example: if "UserService" is mentioned
            directly_injected_service_names.append("UserService")
        # Add more robust extraction if needed, e.g., using regex if the format is predictable, or another LLM call.

        if not directly_injected_service_names:
            print("Could not reliably identify specific service names from the direct injection context. Skipping second-level injection checks for now.")
        
        for service_name in directly_injected_service_names:
            sub_q_service_dependencies_text = f"What services does the {service_name} inject?" # Dynamic sub-question
            context_service_dependencies = get_context_for_question(sub_q_service_dependencies_text, chain)
            if "Error obtaining information" not in context_service_dependencies and "No specific information found" not in context_service_dependencies:
                 collected_contexts_for_synthesis.append(f"Information about services injected by {service_name}: {context_service_dependencies}")
    else:
        print(f"Could not get initial context for direct injections of IfAuthenticatedDirective. Context received: {context_direct_injections}")


    # 3. Synthesize the final answer using the collected contexts
    if collected_contexts_for_synthesis:
        final_context_str = "\n\n".join(collected_contexts_for_synthesis)
        
        SYNTHESIS_PROMPT_TEMPLATE = """You are an expert AI assistant.
Your task is to answer the 'Original Question' based *only* on the provided 'Context from sub-questions'.
Organize your answer clearly. If the context is insufficient or doesn't directly answer the original question, state that clearly. Do not invent information not present in the context.

Context from sub-questions:
---
{collected_sub_contexts}
---

Original Question: {original_question}

Final Answer:"""

        synthesis_prompt = PromptTemplate(
            input_variables=["collected_sub_contexts", "original_question"],
            template=SYNTHESIS_PROMPT_TEMPLATE
        )

        # Create a simple LLMChain for the synthesis step
        synthesis_chain = LLMChain(llm=llm, prompt=synthesis_prompt, verbose=True)

        display(Markdown(f"#### Synthesizing Final Answer for: \"{complex_question_2}\""))
        print(f"\n>>> Synthesizing final answer. Combined context length: {len(final_context_str)} chars.")
        try:
            final_answer_response = synthesis_chain.invoke({
                "collected_sub_contexts": final_context_str,
                "original_question": complex_question_2
            })
            # LLMChain output is a dict, the answer is usually in the 'text' key
            final_answer = final_answer_response.get('text', "Could not synthesize an answer from the LLM.")
            display(Markdown(f"**Synthesized LLM Answer:** {final_answer}"))
        except Exception as e:
            print(f"!!! Error during final answer synthesis for complex question: {e}")
            display(Markdown(f"<font color='red'>Error during synthesis: {e}</font>"))
    else:
        display(Markdown("<font color='orange'>**No contexts were collected for the complex question, cannot synthesize a final answer.**</font>"))

else:
    display(Markdown("<font color='orange'>**Skipping Complex Question Decomposition example as prerequisites (LangChain components, Neo4j/Ollama connection, or 'chain' object) are not met.**</font>"))

## 5.2 Handling Complex Questions with Decomposition (Chain of Thought)


---

### Processing Complex Question: What services are ultimately used by the 'IfAuthenticatedDirective', considering services it injects directly, and then services that THOSE directly injected services might inject (one further level deep)?


>>> Executing sub-question for context: "Which services are directly injected into the IfAuthenticatedDirective?"


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (d:Directive {name: 'IfAuthenticatedDirective'})-[:INJECTS]->(s:Service) RETURN s.name, labels(s)
Full Context:
[{'s.name': 'UserService', 'labels(s)': ['Service', 'AngularEntity']}]

> Finished chain.
<<< Context from sub-question: "UserService."

>>> Executing sub-question for context: "What services does the UserService inject?"


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (s:Service {name: 'UserService'})-[:INJECTS]->(dep) RETURN dep.name, labels(dep)
Full Context:
[{'dep.name': 'Router', 'labels(dep)': ['ExternalOrUnresolved']}, {'dep.name': 'JwtService', 'labels(dep)': ['Service', 'AngularEntity']}, {'dep.name': 'HttpClient', 'labels(dep)': ['ExternalOrUnresolved']}]

> Finished chain.
<<< Context from sub-question: "Router, JwtService, HttpClient."


/tmp/ipykernel_8842/1341281640.py:88: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  synthesis_chain = LLMChain(llm=llm, prompt=synthesis_prompt, verbose=True) # llm is from Cell A


#### Synthesizing Final Answer for: "What services are ultimately used by the 'IfAuthenticatedDirective', considering services it injects directly, and then services that THOSE directly injected services might inject (one further level deep)?"


>>> Synthesizing final answer. Combined context length: 163 chars.


> Entering new LLMChain chain...
Prompt after formatting:
You are an expert AI assistant.
Your task is to answer the 'Original Question' based *only* on the provided 'Context from sub-questions'.
Organize your answer clearly. If the context is insufficient or doesn't directly answer the original question, state that clearly. Do not invent information not present in the context.

Context from sub-questions:
---
Information about direct injections for IfAuthenticatedDirective: UserService.

Information about services injected by UserService: Router, JwtService, HttpClient.
---

Original Question: What services are ultimately used by the 'IfAuthenticatedDirective', considering services it injects directly, and then services that THOSE directly injected services might inject (one further level deep)?

Final Answer:

> Finished chain.


**Synthesized LLM Answer:** Based on the provided context, here is the answer to the original question:

**Directly Injected Services:** The `IfAuthenticatedDirective` injects the following services directly:

1. `UserService`

**Services Injected by Directly Injected Services:** Since `UserService` is injected into `IfAuthenticatedDirective`, let's look at what services `UserService` injects:

1. `Router`
2. `JwtService`
3. `HttpClient`

These services are indirectly used by the `IfAuthenticatedDirective` through its direct injection of the `UserService`.

**Final Answer:** The `IfAuthenticatedDirective` ultimately uses the following services:

1. `UserService`
2. `Router`
3. `JwtService`
4. `HttpClient`

Note that this answer only considers one level deep, as per the original question's request.